In [56]:
# Import dependencies.
import pandas as pd
import numpy as np

In [57]:
# Read the data file and store it in a pandas DataFrame.
school_data_df = pd.read_csv("Resources/schools_complete.csv")
student_data_df = pd.read_csv("Resources/students_complete.csv")

In [58]:
# Determine if there are any missing values.

# school_data_df.count()
# student_data_df.count()

In [59]:
# To get the total number of empty rows

# student_data_df.isnull().sum()
# student_data_df.notnull().sum()

In [60]:
# Determine data types.

# school_data_df.dtypes
# student_data_df.dtypes

### Correct the Student Names

In [61]:
# Add each prefixes and suffixes to remove to a list.
prefixes_suffixes = ["Dr. ", "Ms. ", "Mr. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate the prefeixes suffixes and replace.
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"", regex=True)

student_data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [62]:
# Replace the math / reading scores from 9th grade at Thomas High School with NaN.
student_data_df.loc[(student_data_df["grade"] == "9th") & 
                    (student_data_df["school_name"] == "Thomas High School"),
                    "math_score"] = np.nan

student_data_df.loc[(student_data_df["grade"] == "9th") & 
                    (student_data_df["school_name"] == "Thomas High School"),
                    "reading_score"] = np.nan

student_data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66.0,79.0
1,1,Victor Smith,M,12th,Huang High School,94.0,61.0
2,2,Kevin Rodriguez,M,12th,Huang High School,90.0,60.0
3,3,Richard Scott,M,12th,Huang High School,67.0,58.0
4,4,Bonnie Ray,F,9th,Huang High School,97.0,84.0


In [63]:
student_data_df.tail()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
39165,39165,Donna Howard,F,12th,Thomas High School,99.0,90.0
39166,39166,Dawn Bell,F,10th,Thomas High School,95.0,70.0
39167,39167,Rebecca Tanner,F,9th,Thomas High School,NaN,NaN
39168,39168,Desiree Kidd,F,10th,Thomas High School,99.0,90.0
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95.0,75.0


### District Summary

In [64]:
# Combine dataframes.
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])
school_data_complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66.0,79.0,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94.0,61.0,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90.0,60.0,0,District,2917,1910635
3,3,Richard Scott,M,12th,Huang High School,67.0,58.0,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97.0,84.0,0,District,2917,1910635


In [65]:
# Total number of students / schools.
student_count = school_data_complete_df["Student ID"].count()
school_count = len(school_data_complete_df["school_name"].unique())

In [66]:
# Total budget.
total_budget = school_data_df["budget"].sum()

In [67]:
# Average math / reading score.
average_math_score = school_data_complete_df["math_score"].mean()
average_reading_score = school_data_complete_df["reading_score"].mean()

In [68]:
# Number of ninth grade students at Thomas High School.
no_grade = school_data_complete_df.loc[(school_data_complete_df["grade"]=="9th") & 
                                       (school_data_complete_df["school_name"]=="Thomas High School"), 
                                       ["student_name"]].count()

# New total student count.
new_student_count = float(student_count) - float(no_grade)

In [69]:
# Students who are passing math / reading in a new DataFrame.
passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]
passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]

# Students who are passing both math and reading in a new DataFrame.
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & 
                                               (school_data_complete_df["reading_score"] >= 70)]

In [70]:
# Number of students passing math / reading.
passing_math_count = passing_math["student_name"].count()
passing_reading_count = passing_reading["student_name"].count()

# Number of students passing both math and reading.
passing_math_reading_count = passing_math_reading["student_name"].count()

In [71]:
# Percentage of passed math / reading.
passing_math_percentage = passing_math_count / float(new_student_count) * 100
passing_reading_percentage = passing_reading_count / float(new_student_count) * 100

# Percentage overall passing.
overall_percentage = passing_math_reading_count / float(new_student_count) * 100

In [72]:
# Adding a list of values with keys to create a new DataFrame.
district_summary_df = pd.DataFrame(
                            [{"Total Schools": school_count,
                              "Total Students": student_count,
                              "Total Budget": total_budget,
                              "Average Math Score": average_math_score,
                              "Average Reading Score": average_reading_score,
                              "% Passing Math": passing_math_percentage,
                              "% Passing Reading": passing_reading_percentage,
                              "% Overall Passing": overall_percentage}])

In [73]:
# Formatting
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.9,81.9,74.8,85.7,64.9


##  School Summary

In [74]:
# Determine the school type.
per_school_types = school_data_df.set_index(["school_name"])["type"]

In [75]:
# Calculate the total student per school count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

In [76]:
# Calculate the total school budget.
per_school_budget = school_data_df.set_index(["school_name"])["budget"]

In [77]:
# Calculate the per capita spending.
# (per_school_budget and per_school_counts are Series, both data types are int64 with the same index.)
per_school_capita = per_school_budget / per_school_counts

In [78]:
# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

In [79]:
# Calculate the passing scores.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]

per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

per_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & 
                                                   (school_data_complete_df["reading_score"] >= 70)]

In [80]:
# Calculate the number of students passing by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]

per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

In [81]:
# Calculate the percentage of passing rate per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100

per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [82]:
# Adding a list of values with keys to create a new DataFrame.
per_school_summary_df = pd.DataFrame({
                             "School Type": per_school_types,
                             "Total Students": per_school_counts,
                             "Total School Budget": per_school_budget,
                             "Per Student Budget": per_school_capita,
                             "Average Math Score": per_school_math,
                             "Average Reading Score": per_school_reading,
                             "% Passing Math": per_school_passing_math,
                             "% Passing Reading": per_school_passing_reading,
                             "% Overall Passing": per_overall_passing_percentage})

In [83]:
# Formatting.
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


In [41]:
# Number of 10th-12th graders from Thomas High School.
THS_students = school_data_complete_df.loc[(school_data_complete_df["grade"]!="9th") & 
                                           (school_data_complete_df["school_name"]=="Thomas High School"), 
                                           ["student_name"]].count()

In [84]:
# Get all the students passing math / reading from THS.
passing_math = school_data_complete_df.loc[(school_data_complete_df["math_score"] >= 70) & 
                                           (school_data_complete_df["school_name"] == "Thomas High School")]

passing_reading = school_data_complete_df.loc[(school_data_complete_df["reading_score"] >= 70) & 
                                              (school_data_complete_df["school_name"] == "Thomas High School")]

# Get all the students passing both math and reading from THS.
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & 
                                               (school_data_complete_df["reading_score"] >= 70) & 
                                               (school_data_complete_df["school_name"] == "Thomas High School")]

In [85]:
# Calculate the percentage of 10th-12th grade students from THS. 
passing_math_count = passing_math["student_name"].count()
passing_reading_count = passing_reading["student_name"].count()
passing_math_reading_count = passing_math_reading["student_name"].count()

passing_math_percentage = passing_math_count / float(THS_students) * 100
passing_reading_percentage = passing_reading_count / float(THS_students) * 100
overall_percentage = passing_math_reading_count / float(THS_students) * 100

In [86]:
# Replace the passing percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc["Thomas High School", "% Passing Math"] = passing_math_percentage
per_school_summary_df.loc["Thomas High School", "% Passing Reading"] = passing_reading_percentage
per_school_summary_df.loc["Thomas High School", "% Overall Passing"] = overall_percentage

per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


## High and Low Performing Schools 

In [87]:
# Sort and show top five schools.
top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.350937,83.896082,93.185690,97.018739,90.630324
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [88]:
# Sort and show bottom five schools.
bottom_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=True)
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math and Reading Scores by Grade

In [90]:
# Create a grade level DataFrames.
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]

tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]

eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]

twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

In [93]:
# Group each grade level DataFrame by the school name for the average math / reading score.
ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"] 

In [94]:
# Combine each grade level Series for average math / reading scores by school into a single DataFrame.
math_scores_by_grade = pd.DataFrame({
                               "9th": ninth_grade_math_scores,
                               "10th": tenth_grade_math_scores,
                               "11th": eleventh_grade_math_scores,
                               "12th": twelfth_grade_math_scores})

reading_scores_by_grade = pd.DataFrame({
                                  "9th": ninth_grade_reading_scores,
                                  "10th": tenth_grade_reading_scores,
                                  "11th": eleventh_grade_reading_scores,
                                  "12th": twelfth_grade_reading_scores})

In [95]:
# Formatting.
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.1f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.1f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.1f}".format)

# Make sure the columns are in the correct order.
math_scores_by_grade = math_scores_by_grade[["9th", "10th", "11th", "12th"]]

# Remove the index name.
math_scores_by_grade.index.name = None

# Display the DataFrame.
math_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4


In [96]:
# Formatting.
reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:,.1f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:,.1f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:,.1f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:,.1f}".format)

# Make sure the columns are in the correct order.
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]

# Remove the index name.
reading_scores_by_grade.index.name = None

# Display the data frame.
reading_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0


## Scores by School Spending

In [97]:
# Get the descriptive statistics for the per_school_capita.
per_school_capita.describe()

count     15.000000
mean     620.066667
std       28.544368
min      578.000000
25%      591.500000
50%      628.000000
75%      641.500000
max      655.000000
dtype: float64

In [98]:
# Cut the per_school_capita into the spending ranges.
spending_bins = [0, 585, 630, 645, 675]

per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()

(0, 585]      4
(585, 630]    4
(630, 645]    4
(645, 675]    3
dtype: int64

In [99]:
# Establish the spending bins and group names. (Name each range)
spending_bins = [0, 585, 630, 645, 675]

group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

In [100]:
# Categorize spending based on the bins.
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)
per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-629
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$584
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-644
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-644
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-629


In [101]:
# Calculate averages for the desired columns.
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [102]:
# Assemble into DataFrame.
spending_summary_df = pd.DataFrame({
                              "Average Math Score" : spending_math_scores,
                              "Average Reading Score": spending_reading_scores,
                              "% Passing Math": spending_passing_math,
                              "% Passing Reading": spending_passing_reading,
                              "% Overall Passing": overall_passing_spending})

In [103]:
# Formatting.
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)

spending_summary_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,81.9,83.2,87,93,81
$630-644,78.5,81.6,73,84,63
$645-675,77.0,81.0,66,81,54


## Scores by School Size

In [104]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [105]:
# Categorize spending based on the bins.
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)

In [106]:
# Calculate averages for the desired columns.
size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]

size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

In [107]:
# Assemble into DataFrame.
size_summary_df = pd.DataFrame({
                          "Average Math Score" : size_math_scores,
                          "Average Reading Score": size_reading_scores,
                          "% Passing Math": size_passing_math,
                          "% Passing Reading": size_passing_reading,
                          "% Overall Passing": size_overall_passing})

In [108]:
# Formatting.
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)
size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)
size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)
size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)

size_summary_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-2000),83.4,83.9,94,97,91
Large (2000-5000),77.7,81.3,70,83,58


## Scores by School Type

In [109]:
# Calculate averages for the desired columns.
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]

type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

In [111]:
# Assemble into DataFrame.
type_summary_df = pd.DataFrame({
                          "Average Math Score" : type_math_scores,
                          "Average Reading Score": type_reading_scores,
                          "% Passing Math": type_passing_math,
                          "% Passing Reading": type_passing_reading,
                          "% Overall Passing": type_overall_passing})

In [112]:
# Formatting.
type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)
type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)
type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)
type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)
type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)

type_summary_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54
